In [1]:
import pandas as pd
import numpy as np

from konlpy.tag import Kkma, Komoran, Okt, Hannanum
import os
import re

In [2]:
def clean_str(text):
    text = str(text)
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)

    text = re.sub('\n', '.', string=text)

    pattern = '[^\w\s\n?0-9]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    text = re.sub('[-=+,#/\:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', string=text)
    
    text = text.strip()
    text = text.replace('    ', ' ')
    text = text.replace('   ', ' ')
    text = text.replace('  ', ' ')
    
    
    # for n in prod_name:
    #     text = text.replace(n, '')

    # pattern_punctuation = re.compile(r'[^\w\s]')
    # text = pattern_punctuation.sub('', text)
    return text 

In [3]:
# prod_list = os.listdir("./data/")

# # 0을 for문으로 돌면서 상품 하나 돌 때마다 prod_name 생성하기
# for name in prod_list:
#     # name = prod_list[0]
#     prod_name = name.split(",")[0].split(" ")
#     set(prod_name)

In [4]:
def make_review(test,prod_name):
    okt = Okt()
    new_review = []
    for review in test:
        text = clean_str(review)
        for name in prod_name:
            text = text.replace(name, "")
        

        text = text.replace("  ", " ")
        test2 = text.split(' ')
#         text2 = [t for t in test2 if t not in prod_name]

        test2 = " ".join(test2)
        test2 = okt.morphs(test2, stem= True)

        # 조사
        stopwords1 = ['도', '만', '을', '를', '이', '가', '은', '는', 'null']
        # 지시대명사
        stopwords2 = ['이', '그', '저', 'null']
        # 그 외
        stopwords3 = ['등', '편', '번', '시', '나', '씩', '꼭', 
                      '으로', '로', '걸', '하다', '수', '와', '과', 
                      '들', '니', '또', '면', '구', '요', '것', '분', '해', '땐', '?', '건', 
                      '순', '때', '제', '에', 'null', '입니다', '']
        
        test2 = [t for t in test2 if t not in stopwords1]
        test2 = [t for t in test2 if t not in stopwords2]
        test2 = [t for t in test2 if t not in stopwords3]
        
        test2 = [t for t in test2 if t not in prod_name]
        
        test3 = []
        while len(prod_name)>0:
            stop = prod_name.pop(0)
#             print(stop)
            for word in test2:
                if word not in stop:
#                     print(stop, word)
                    test3.append(word)
#                     print(test3)
            test2 = test3
            test3 = []

        new_review.append(test2)
    return new_review

In [5]:

# kkma = Kkma()
# komoran = Komoran()
# hannanum = Hannanum()
prod_list = os.listdir("./NoEvi/")
review_content = []
for name in prod_list[0:300]:
#     name = prod_list[0]
    prod_name = name.split(",")[0].split(" ")
    set(prod_name)
    
    df = pd.read_excel(f"./NoEvi/{name}")
    test = df["리뷰 내용"].values.tolist() # name 이라는 상품의 모든 리뷰가 담겨있음
    new_review = make_review(test, prod_name)
    review_content += new_review
    
# name = prod_list[0]
# prod_name = name.split(",")[0].split(" ")
# set(prod_name)

# df = pd.read_excel(f"./data/{name}")
# test = df["리뷰 내용"].values.tolist() # name 이라는 상품의 모든 리뷰가 담겨있음
# new_review = make_review(test,prod_name)




In [27]:
len(review_content)

17625

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
# NoEvinue
text = []
for i in range(len(review_content)):
    test = review_content[i]
#     test.to_list()
#     test = test.split(",")
    test = " ".join(test)
    text.append(test)

In [9]:
tfidfv = TfidfVectorizer().fit(text)
print(tfidfv.transform(text).toarray())
print()
print(tfidfv.vocabulary_)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

{'진쨔': 16935, '인생': 15060, '찾다': 17476, '니당다': 4720, '회사': 20615, '동료': 5636, '언니': 12881, '어느': 12697, '사다': 9926, '사용': 9996, '해보다': 20065, '쓰윽': 11907, '주다': 16446, '반하다': 8386, '바로': 8203, '쿠팡': 18315, '에서': 13100, '구매': 3020, '진짜': 16932, '물건': 7847, '이에요': 14901, '좋다': 16383, '갖다': 1998, '달다': 4982, '무지막지': 7776, '크다': 18368, '퍼프덕분': 19201, '몇번': 7492, '거리': 2185, '얼굴': 12901, '호다닥': 20400, '바르다': 8214, '있다': 15349, '퍼프': 19200, '앞쪽': 12481, '뾰족': 9854, '물방': 7875, '모양': 7540, '되어다': 5702, '콧볼': 18290, '세심': 10697, '쿨링': 18332, '효과': 20629, '알로': 12370, '에가': 13029, '41': 576, '가다': 1625, '함유': 19983, '나오다': 4035, '산뜻하다': 10094, '가볍다': 1704, '발리다': 8438, '시원하다': 11498, '때문': 6145, '톡톡': 18752, '두드리다': 5739, '답답하다': 5056, '전혀': 15943, '없다': 12992, '화장': 20522, '파운데이션': 19045, '이나': 14686, '비비': 9572, '뭔가': 7956, '모르다': 7523, '

In [10]:
reverse_ = {v:k for k,v in tfidfv.vocabulary_.items()}

col = [0]*len(reverse_)
for i in range(len(reverse_)):
    col[i] = reverse_[i]

In [11]:
# 데이터 프레임으로 바꿔주기
data = tfidfv.transform(text).toarray()
tfidf_matrix = pd.DataFrame(data, columns = col)

In [12]:
cols = tfidf_matrix.columns

In [13]:
tfidf_matrix.head(1)

,00,000,00000000001,00001,0005,000원,005,01,01024072031,010원,...,힘쓰다,힘안,힘없다,힘유,힘주다,힙듭니,ｊヽ,ｦｦｦ,ﾉﾞ,ﾟﾟ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# 행 하나만 해보면..
dict = {}
for col in cols:
    dict[col] = tfidf_matrix.head(1)[col][0]
sorted(dict.items(), key=lambda x: x[1], reverse=True)

In [52]:
sorted(dict.items(), key=lambda x: x[1], reverse=True)

[('귀엽다', 0.3176494467104484),
 ('톤업', 0.26209805472183684),
 ('진짜', 0.21940655389720562),
 ('노랑', 0.1703069212449481),
 ('들어주다', 0.1703069212449481),
 ('모양', 0.1703069212449481),
 ('함께', 0.1443588648092453),
 ('답답하다', 0.13919577641320938),
 ('되어다', 0.1338801659951586),
 ('좋다', 0.13249833401868188),
 ('분들', 0.13104902736091842),
 ('전혀', 0.12472991786723048),
 ('완전', 0.12203086849520291),
 ('은은하다', 0.12203086849520291),
 ('바르다', 0.11870246821281337),
 ('시원하다', 0.10970327694860281),
 ('있다', 0.10506647918571815),
 ('화장', 0.10240192155318809),
 ('되다', 0.09520150823482297),
 ('피부', 0.09376144230562951),
 ('해보다', 0.09361877303549175),
 ('효과', 0.08767978447613167),
 ('촉촉하다', 0.0854720239832008),
 ('41', 0.08515346062247404),
 ('bbbb', 0.08515346062247404),
 ('남성', 0.08515346062247404),
 ('남자친구', 0.08515346062247404),
 ('년차', 0.08515346062247404),
 ('동료', 0.08515346062247404),
 ('무지막지', 0.08515346062247404),
 ('물광', 0.08515346062247404),
 ('물방', 0.08515346062247404),
 ('뭐해', 0.08515346062247404

In [81]:
# evinue와 noevinue 에서 중요한 단어를 찾는 거니까 컬럼 별로 tf-idf 값 더하기
dict = {}
for col in cols:
    dict[col] = tfidf_matrix[col].sum()

In [54]:
dict

{'00': 0.08292209978861351,
 '01': 0.07740754127221615,
 '03': 0.36723867286475065,
 '05': 0.08443906661114253,
 '050원': 0.07432898044508326,
 '06': 0.1400005088409295,
 '08': 0.14758050845124007,
 '10': 0.4980708188866334,
 '100': 0.29863241109490424,
 '10000': 0.07633156012250215,
 '10일': 0.14900417098561447,
 '11': 0.4616256513812861,
 '112원': 0.08443906661114253,
 '11시': 0.07007710822052551,
 '11월': 0.22657658333464764,
 '12시': 0.07007710822052551,
 '12시간': 0.16317820811826192,
 '13': 0.2886056630438215,
 '13일': 0.08199271895977847,
 '14': 0.6121661413120767,
 '15': 0.09193777793375796,
 '17': 0.09509348705156367,
 '18': 0.15986746454313527,
 '18130원': 0.1285816347839931,
 '19': 0.07740754127221615,
 '19일': 0.08793695445627629,
 '1년': 0.18319133088772394,
 '1만원': 0.16530747663216924,
 '1시간': 0.1203867362497792,
 '1일': 0.08793695445627629,
 '20': 0.4137085876874613,
 '2019년': 0.6347635341629494,
 '2020': 0.08432815338009882,
 '2021': 0.15109174239532278,
 '2022': 0.27628930365443094

In [55]:
sorted(dict.items(), key=lambda x: x[1], reverse=True)

[('바르다', 17.557392019683295),
 ('좋다', 17.31293945475131),
 ('있다', 13.399722821462174),
 ('보다', 11.355474764209777),
 ('같다', 11.186134396096147),
 ('크림', 11.011995420569562),
 ('않다', 10.826422585965553),
 ('사용', 10.691748450610053),
 ('제품', 10.616008569542801),
 ('없다', 10.536225125626142),
 ('피부', 10.388048080632156),
 ('쓰다', 10.048134463675257),
 ('써다', 9.819362209231018),
 ('스틱', 9.2673474501341),
 ('자다', 8.855288556980767),
 ('되다', 8.490631538681068),
 ('구매', 8.413937550250619),
 ('느낌', 8.02063276966858),
 ('너무', 7.87329751806133),
 ('가격', 7.6748752541066985),
 ('얼굴', 6.960933494174066),
 ('이다', 6.664733673388691),
 ('보습', 6.5816899457209725),
 ('발라', 6.231821483469392),
 ('발림', 5.707561204812839),
 ('촉촉하다', 5.549578823978562),
 ('아니다', 5.4756039796824245),
 ('괜찮다', 5.191804640856905),
 ('모르다', 5.177223154246008),
 ('ahc', 5.164310532190944),
 ('쿠션', 5.163937531442285),
 ('편하다', 5.134836962627943),
 ('사다', 4.996503668707769),
 ('하고', 4.790923420560068),
 ('정도', 4.7292600198034),
 ('들

In [82]:
dict

{'00': 0.06725619854725444,
 '000': 1.8511176957588762,
 '000원': 2.7200712839624357,
 '00시': 0.16008718089899882,
 '01': 1.347557958079831,
 '01월': 0.4125338898110903,
 '01일': 0.16560437573063844,
 '02': 0.8337195893289848,
 '020원': 0.09967741613178299,
 '024원': 0.08516078646918526,
 '02월': 0.3429818355948375,
 '02일': 0.06286315304964618,
 '03': 0.866830969500586,
 '035': 0.10320483502698316,
 '03월': 0.12572630609929236,
 '03일': 0.17272039941364498,
 '04': 0.6676262065334293,
 '04월': 0.30360094627494416,
 '05': 0.68182152653087,
 '050원': 0.1437675296749169,
 '05월': 0.5980624902371814,
 '05일': 0.12502871104163102,
 '06': 0.44958149374228107,
 '060원': 0.09420167241432444,
 '06월': 0.1253385457242028,
 '06일': 0.1000931253402831,
 '07': 1.2880465780732735,
 '075': 0.17377600084367423,
 '07분': 0.16008718089899882,
 '08': 0.9549525653638979,
 '080': 0.16008718089899882,
 '08월': 0.21924677402961176,
 '08일': 0.7089030218323289,
 '09': 1.3602030010608428,
 '090원': 0.1005488667893219,
 '09월': 0.2

In [83]:
sorted(dict.items(), key=lambda x: x[1], reverse=True)

[('좋다', 227.22701361175888),
 ('피부', 163.9662699171092),
 ('있다', 141.69501166166046),
 ('제품', 141.0820924874295),
 ('사용', 140.39377985440854),
 ('바르다', 132.1719369246269),
 ('같다', 119.57538875388116),
 ('이다', 109.36965467454667),
 ('보다', 108.93912755670917),
 ('되다', 105.3657482606096),
 ('쓰다', 103.99129268454865),
 ('않다', 98.80421046762376),
 ('크림', 94.20150677556853),
 ('구매', 93.64971602243664),
 ('없다', 91.70118601360865),
 ('자다', 86.49196785796059),
 ('써다', 85.9559459121431),
 ('너무', 84.5190309961026),
 ('설화수', 82.8959644234256),
 ('느낌', 81.61301076085618),
 ('촉촉하다', 79.73306517787624),
 ('받다', 78.76515950245205),
 ('쿠팡', 74.97094847614477),
 ('선물', 72.5226332346559),
 ('상품', 68.76368565316685),
 ('가격', 65.82799053545432),
 ('들다', 63.99661638705449),
 ('배송', 62.60689054664835),
 ('후기', 62.14687572070677),
 ('체험', 60.94523849546171),
 ('보습', 60.09443488911877),
 ('화장품', 59.60066268001259),
 ('얼굴', 56.55716589302353),
 ('아니다', 55.07968767283177),
 ('되어다', 53.75829154020188),
 ('에센스', 5

In [84]:
evi =[('좋다', 227.22701361175888),
 ('피부', 163.9662699171092),
#  ('있다', 141.69501166166046),
#  ('제품', 141.0820924874295),
 ('사용', 140.39377985440854),
 ('바르다', 132.1719369246269),
#  ('같다', 119.57538875388116),
#  ('이다', 109.36965467454667),
#  ('보다', 108.93912755670917),
 ('되다', 105.3657482606096),
#  ('쓰다', 103.99129268454865),
#  ('않다', 98.80421046762376),
 ('크림', 94.20150677556853),
 ('구매', 93.64971602243664),
#  ('없다', 91.70118601360865),
#  ('자다', 86.49196785796059),
#  ('써다', 85.9559459121431),
#  ('너무', 84.5190309961026),
#  ('설화수', 82.8959644234256),
 ('느낌', 81.61301076085618),
 ('촉촉하다', 79.73306517787624),
 ('받다', 78.76515950245205),
#  ('쿠팡', 74.97094847614477),
 ('선물', 72.5226332346559),
#  ('상품', 68.76368565316685),
 ('가격', 65.82799053545432),
 ('들다', 63.99661638705449),
 ('배송', 62.60689054664835),
 ('후기', 62.14687572070677),
 ('체험', 60.94523849546171),
 ('보습', 60.09443488911877),
 ('화장품', 59.60066268001259),
 ('얼굴', 56.55716589302353),
#  ('아니다', 55.07968767283177),
#  ('되어다', 53.75829154020188),
 ('에센스', 53.483545942275335),
#  ('이라', 53.245091422983585),
#  ('에서', 51.78788245062118),
#  ('사다', 51.731416830177935),
 ('건조하다', 51.41630491468637),
#  ('하고', 50.34746322694467),
#  ('인데', 49.536632357372234),
 ('작성', 47.81947133258626),
#  ('오다', 47.747311481177),
 ('수분', 47.47798457790694),
 ('스킨', 47.31631929073474),
 ('많이', 46.83702524061313),
 ('제공', 46.56581849264377),
#  ('자음', 46.50713113193028),
 ('흡수', 46.32734794437742),
 ('무료', 45.949764787625426),
 ('이벤트', 44.94755183755689),
 ('발라', 43.533449752888075),
#  ('생각', 43.48257480801841),
 ('구입', 42.859320596293784),
 ('없이', 41.77487439032143),
 ('저렴하다', 41.72195945855514),
 ('만족하다', 41.61771145562531),
 ('그렇다', 41.05121123644582),
 ('포장', 40.261230172167025),
 ('맞다', 40.23268761092372),
#  ('ml', 40.0178577850739),
#  ('까지', 39.52495257501977),
 ('효과', 39.37306375491969),
 ('다른', 38.541678589076895),
#  ('헤라', 38.44930144440379),
 ('나다', 38.087421222007976),
 ('해보다', 37.39079998460312),
#  ('정도', 37.30507340323861),
 ('로션', 37.09284090864877),
 ('탄력', 37.004243324758434),
#  ('쓸다', 36.74911018255189),
 ('샘플', 36.295376609931196),
 ('아주', 36.00061346169109),
 ('괜찮다', 35.814078037856824),
 ('정말', 35.62511060773504),
 ('엄마', 35.02686311446814),
#  ('이에요', 34.87740162831789),
 ('많다', 34.626224865807195),
#  ('에는', 34.31542383557277),
 ('좋아하다', 33.64231633872808),
 ('드리다', 33.317527155927586),
 ('성분', 33.231792517941656),
 ('건성', 33.170556129071116),
 ('부드럽다', 32.739662440992404),
 ('선크림', 32.341361559137965),
 ('트러블', 32.298269746904424),
#  ('조금', 32.06802108557629),
 ('화장', 31.976158396221166),
 ('넘다', 31.69141005804142),
 ('주문', 31.6346001614594),
 ('빠르다', 31.506539700109332),
 ('수분크림', 31.168435757583257),
 ('그냥', 31.100362738693374),
 ('추천', 31.034851511375038),
 ('가볍다', 30.940486200139866),
 ('아침', 30.908742484317838),
 ('냄새', 30.776796991798548),
 ('처음', 30.50066873970983),
 ('해주다', 30.48444175255166),
 ('프리', 30.045340370477646),
 ('처럼', 30.012921885837798),
 ('모르다', 29.822265646476822),
#  ('보고', 29.53386222228719),
#  ('메라', 29.357238897295183),
#  ('발림', 29.111496466572817),
 ('자극', 29.033046277981963),
 ('싶다', 29.01198250106246),
 ('용량', 28.92932363366101),
 ('비싸다', 28.723075285010662),
 ('하나', 28.62903096684412),
 ('주다', 28.531248445201285),
 ('박스', 28.384327737376424),
#  ('에도', 28.239695333737714),
 ('느껴지다', 28.01171684461557),
 ('느끼다', 27.633439528451156),
 ('은은하다', 27.57862239371468),
 ('바로', 27.553004871890884),
 ('기한', 27.390178103791342),
 ('아이', 27.31470515581149),
 ('재다', 27.30710935795403),
 ('알다', 27.239924836527926),
 ('타입', 27.16861150479839),
 ('이나', 27.097301865086497),
 ('쇼핑', 26.962696585048676),
 ('기초', 26.90072392379276),
 ('여름', 26.79843417770437),
 ('약간', 26.652109885007675),
 ('가다', 26.639931151350222),
 ('믿다', 26.60387750113548),
 ('항상', 26.581669653205733),
 ('기분', 26.469804509408753),
 ('라인', 26.44223979366887),
 ('세트', 26.38575555205684),
 ('워터', 26.296756610345067),
 ('크다', 26.22242060774984),
 ('꾸준하다', 26.14626307109294),
 ('순하다', 25.61064185954398),
 ('같이', 25.600088081344445),
 ('쓰기', 25.587214822776836),
 ('진짜', 25.35103011937417),
 ('한번', 25.075101317063677),
 ('부터', 24.724170658911625),
 ('그래서', 24.64013381961135),
 ('요즘', 24.60976218650484),
 ('역시', 24.2670459638362),
 ('주름', 24.09977543457729),
 ('적다', 23.82334112431736),
 ('유통', 23.75355749183401),
 ('예요', 23.703491033549412),
 ('엄청', 23.44096018599174),
 ('대다', 23.378442518528587),
 ('다음', 23.291417052880117),
 ('계속', 23.253760189405646),
 ('일단', 23.11527134908043),
 ('스며들다', 22.682198260961407),
 ('다르다', 22.60785356778647),
 ('정품', 22.48100195869929),
 ('적임', 22.407486315149253),
 ('이번', 22.159555048426732),
 ('도움', 22.057207151485564),
 ('나오다', 21.857596743312925),
 ('백화점', 21.82008473670169),
 ('이지', 21.755566799688037),
 ('토너', 21.695124362481813),
 ('싸다', 21.691787877562767),
 ('브랜드', 21.663840912457545),
 ('마스크', 21.633330681886388),
 ('그리고', 21.599069777118828),
 ('산뜻하다', 21.582795238006955),
 ('묽다', 21.453210336644332),
 ('겨울', 21.37877651936887),
 ('살짝', 21.307946689237863),
 ('용기', 21.259924100016523),
 ('저녁', 21.207497317263368),
 ('돼다', 20.99783700668518),
 ('50', 20.72385050461919),
 ('생기다', 20.605014958743126),
 ('적당하다', 20.59044995239355),
 ('시키다', 20.425956974406958),
 ('확실하다', 20.337557328821163),
 ('나이', 20.123724077328713),
 ('보이다', 20.027354520651805),
 ('이고', 20.001555469607567),
 ('작다', 19.927574074347767),
 ('원래', 19.855474995171235),
 ('케어', 19.44427434145528),
 ('때문', 19.412908960432468),
 ('지다', 19.392786849989715),
 ('완전', 19.369052129176282),
 ('당기다', 19.283498640427432),
 ('리뷰', 19.2235163823825),
 ('다시', 19.12264521931324),
 ('윤조', 19.112769941187743),
 ('최고', 19.03666434920145),
 ('마음', 19.03260385875732),
 ('뚜껑', 19.021966663846783),
 ('떨어지다', 19.02060884680973),
 ('썬크림', 19.01715624358578),
 ('고민', 18.984527867586536),
 ('부분', 18.931960138397002),
 ('근데', 18.856706253810444),
 ('지금', 18.699911861571735),
 ('별로', 18.644921012400204),
 ('보내다', 18.643376891501),
 ('세안', 18.432294475705106),
 ('편이', 18.316818937139793),
 ('감사하다', 18.2748243263312),
 ('할인', 18.210355063458884),
 ('에요', 18.193760470996992),
 ('진정', 18.145226946069823),
 ('거리', 18.081034553159228),
 ('상태', 17.89506849459645),
 ('사람', 17.838221123353392),
 ('이랑', 17.822714413990784)]

In [85]:
no = [('바르다', 17.557392019683295),
 ('좋다', 17.31293945475131),
#  ('있다', 13.399722821462174),
#  ('보다', 11.355474764209777),
#  ('같다', 11.186134396096147),
 ('크림', 11.011995420569562),
#  ('않다', 10.826422585965553),
 ('사용', 10.691748450610053),
#  ('제품', 10.616008569542801),
#  ('없다', 10.536225125626142),
 ('피부', 10.388048080632156),
#  ('쓰다', 10.048134463675257),
#  ('써다', 9.819362209231018),
 ('스틱', 9.2673474501341),
#  ('자다', 8.855288556980767),
 ('되다', 8.490631538681068),
#  ('구매', 8.413937550250619),
#  ('느낌', 8.02063276966858),
#  ('너무', 7.87329751806133),
 ('가격', 7.6748752541066985),
 ('얼굴', 6.960933494174066),
#  ('이다', 6.664733673388691),
 ('보습', 6.5816899457209725),
#  ('발라', 6.231821483469392),
#  ('발림', 5.707561204812839),
 ('촉촉하다', 5.549578823978562),
#  ('아니다', 5.4756039796824245),
 ('괜찮다', 5.191804640856905),
 ('모르다', 5.177223154246008),
#  ('ahc', 5.164310532190944),
 ('쿠션', 5.163937531442285),
 ('편하다', 5.134836962627943),
#  ('사다', 4.996503668707769),
#  ('하고', 4.790923420560068),
#  ('정도', 4.7292600198034),
#  ('들다', 4.722132654963531),
 ('저렴하다', 4.6218439460229535),
#  ('콜라겐', 4.537506598632065),
#  ('생각', 4.505352894542814),
 ('많이', 4.221461441605375),
 ('효과', 4.1088223189404856),
 ('구입', 4.024362544695146),
 ('크다', 3.9646751819985955),
#  ('이라', 3.9404428894857153),
#  ('오다', 3.9051894829561284),
#  ('재다', 3.8839313562577193),
 ('차단', 3.874530741788881),
 ('건조하다', 3.855334358031221),
 ('그냥', 3.7867129549440204),
 ('적임', 3.620036300995478),
 ('그렇다', 3.6126394148680387),
#  ('되어다', 3.6064316786916195),
#  ('아주', 3.564193554409102),
#  ('약간', 3.5435690311490955),
#  ('발리다', 3.5414950034179733),
#  ('나다', 3.538561903619385),
 ('썬크림', 3.5296723056140906),
 ('계속', 3.410136146105152),
#  ('일단', 3.2861478765199044),
#  ('선크림', 3.281943989790446),
#  ('화장품', 3.25989801607576),
 ('주문', 3.2573939892226123),
 ('묻다', 3.237298317135017),
 ('부드럽다', 3.2167373975352236),
 ('부러지다', 3.2135951702864616),
#  ('에도', 3.2103743920686885),
 ('후기', 3.1718582704100093),
 ('많다', 3.1467069161231978),
#  ('보고', 3.1390265895639438),
#  ('가다', 3.131977157134245),
#  ('에는', 3.116882843924329),
#  ('인데', 3.0997262847189013),
 ('용량', 3.0132246267266507),
 ('조금', 2.9886548227717897),
 ('배송', 2.96487964341108),
 ('건성', 2.963412807377895),
 ('에서', 2.884473937649007),
#  ('까지', 2.8820840374829664),
 ('나오다', 2.7891245164944487),
 ('하나', 2.7707159036143425),
 ('리뷰', 2.7197159427080573),
#  ('상품', 2.7099896500971283),
 ('알다', 2.7063408699112372),
 ('해보다', 2.686266845444024),
 ('거리', 2.6569631806055187),
 ('정말', 2.608652754268937),
 ('버리다', 2.5886487702891796),
 ('주름', 2.5816747578964323),
 ('아직', 2.5754251057911555),
 ('현상', 2.571737975568414),
 ('작다', 2.57099872540992),
#  ('이에요', 2.5691141478642496),
 ('겨울', 2.5523332099658553),
 ('무난', 2.5480748325482763),
 ('자외선', 2.540746034667247),
 ('건조', 2.5354395589263223),
 ('가루', 2.533615662063938),
 ('처음', 2.478574124609702),
 ('가성', 2.4763059824198375),
 ('쓸다', 2.474791754216662),
 ('적다', 2.4088751740885552),
 ('다시', 2.396437527728745),
 ('다니다', 2.3868689795943245),
 ('처럼', 2.3728074474768137),
 ('밀리다', 2.370631425878588),
 ('포장', 2.3640579046667667),
 ('쫀쫀', 2.359743404878437),
 ('근데', 2.3525296545087535),
 ('부분', 2.3461569564701508),
 ('싶다', 2.3324241197501694),
 ('엄청', 2.3321460861214516),
 ('기존', 2.290649055993898),
 ('편이', 2.2853589770372507),
 ('가볍다', 2.2777257531796797),
 ('금방', 2.264904057986989),
 ('아침', 2.2602795834958846),
 ('화장', 2.2558667270429),
 ('다른', 2.246392796459034),
 ('휴대', 2.2451174979720716),
 ('넘다', 2.242904951242508),
 ('진짜', 2.237999840276923),
 ('좋아하다', 2.225437819878995),
 ('이지', 2.2175150190638484),
 ('뚜껑', 2.213185841850329),
 ('매트', 2.2064435344320525),
 ('이상', 2.189860612934389),
 ('늘다', 2.1271341776357118),
 ('몇번', 2.1250646738239065),
 ('부담', 2.1235213940713344),
 ('성도', 2.1205544119195494),
 ('살짝', 2.1144552859109345),
 ('에요', 2.1095095558091197),
 ('쿠팡', 2.102688561016553),
 ('맞다', 2.0935786015248334),
 ('찾다', 2.0861644387346874),
 ('시원하다', 2.086161922536013),
 ('그래서', 2.0844696359158914),
 ('여름', 2.0798112474878767),
 ('요즘', 2.054910741028457),
 ('자주', 2.0439428218307687),
 ('이건', 2.034226454980371),
 ('의사', 2.0317417901567016),
 ('타입', 2.0244213550403947),
 ('아이', 2.02058045538447),
 ('안좋다', 2.0126949248951482),
 ('파란색', 2.0079341339207435),
 ('적당하다', 2.0076049869829453),
 ('떨어지다', 2.0000233119406694),
 ('받다', 1.9489784692477832),
 ('때문', 1.9489514530922851),
 ('역시', 1.936667236434091),
 ('꾸준하다', 1.932193098664045),
 ('해봤다', 1.9320458627845443),
 ('흡수', 1.921790585595724),
 ('심하다', 1.9201798969577024),
 ('별로', 1.917169883541768),
 ('트러블', 1.9103576213316773),
 ('빨리', 1.9075530656131625),
 ('부터', 1.8845082000991393),
 ('추천', 1.8780242001046876),
 ('더블', 1.8643906665208279),
 ('주다', 1.8635795048549164),
 ('하얗다', 1.8341587616339583),
 ('거의', 1.8161609411912913),
 ('그래도', 1.8151692409606446),
 ('보이다', 1.8116707672734331),
 ('50', 1.7953676880433214),
 ('나쁘다', 1.7937711214357064),
 ('느끼다', 1.79150727251249),
 ('지다', 1.789254347893206),
 ('개선', 1.783730332597861),
 ('기분', 1.7598604157747655),
 ('탄력', 1.7585798629483456),
 ('발리', 1.7517181052182118),
 ('이네', 1.7444579890027954),
 ('건지다', 1.7379366845984658),
 ('선물', 1.7368276972913086),
 ('자극', 1.7365714548173194),
 ('좀더', 1.73133862974678),
 ('뽀송하다', 1.7226401655636097),
 ('밀림', 1.7223396269615612),
 ('없이', 1.7216328209420526),
 ('이나', 1.7190811746423669),
 ('싸다', 1.7187810961176373),
 ('한번', 1.7179335991487026),
 ('시간', 1.7141682185723202),
 ('크기', 1.6986258992492191),
 ('만족하다', 1.6911856237708394),
 ('간편하다', 1.682242695362396),
 ('굉장하다', 1.6782794298975077),
 ('도움', 1.6673362200111415),
 ('지만', 1.6650860936144463),
 ('쓰기', 1.6538648813760397),
 ('분기', 1.652505404489938),
 ('수분', 1.652488273478941),
 ('남다', 1.6483153533321224),
 ('편리하다', 1.645789099755761),
 ('바꾸다', 1.6381558136085101),
 ('마스크', 1.6324502524376925),
 ('대비', 1.6149333195421784),
 ('추가', 1.6120919161322274),
 ('당기다', 1.609069621350043),
 ('하지만', 1.604211095414037),
 ('이고', 1.6041185620817582),
 ('무엇', 1.5837907669193096),
 ('커서', 1.5816138028126174),
 ('바로', 1.5721249812325624),
 ('톤업', 1.565358272439194),
 ('유통', 1.549187610727654),
 ('딱딱하다', 1.5455109539323213),
 ('쉴드', 1.5386184654718182),
 ('보다는', 1.524081504329934),
 ('기한', 1.5205239458656248),
 ('그리고', 1.5145789424290594),
 ('마음', 1.5079705046979268),
 ('이번', 1.507184816106766),
 ('브랜드', 1.5044873780760775),
 ('기능', 1.4953500124289842),
 ('보통', 1.494118538807649),
 ('믿다', 1.4927055989103208),
 ('빼다', 1.4906318476830165),
 ('형태', 1.4889985516644677),
 ('인거', 1.487839306802539),
 ('전이', 1.482145630856765),
 ('사서', 1.473781526362765),
 ('워낙', 1.4703707380436382),
 ('빠르다', 1.4656641588995263),
 ('듬뿍', 1.462072791423303),
 ('야하다', 1.460659535637369),
 ('기초', 1.4566062259850843),
 ('팩트', 1.4535579231963978),
 ('같이', 1.4448164577481817),
 ('비싸다', 1.4425613308222844),
 ('나이', 1.4413088519481372),
 ('엄마', 1.4402331584529082),
 ('싫다', 1.4401459275030963),
 ('착하다', 1.4370846177885086),
 ('파랑', 1.4338717628735234),
 ('빨간색', 1.4331845416777833),
 ('지금', 1.4301424550887547),
 ('해주다', 1.4270994975166615),
 ('사람', 1.4226615729007597),
 ('보내다', 1.4224121150812685),
 ('로션', 1.406953534081386),
 ('퍼프', 1.391945381203429),
 ('오래', 1.388914692071888),
 ('성분', 1.385161184862903),
 ('신경', 1.3783300228489923),
 ('평소', 1.3703572843308458),
 ('케이스', 1.3689859736101595),
 ('이용', 1.3687925697465624),
 ('미백', 1.356060861293582),
 ('다만', 1.351751511133802),
 ('귀찮다', 1.3436715242099841),
 ('내추럴', 1.342565615515999),
 ('시키다', 1.333923770228591),
 ('유지', 1.3209248315645807),
 ('뻑뻑', 1.3139912182475122),
 ('어보', 1.3133196900118103),
 ('거울', 1.304368124611884),
 ('로켓', 1.304272310799689),
 ('하루', 1.2985367826227963),
 ('잘쓰다', 1.2979096051586565),
 ('단점', 1.2952373144092033),
 ('예민하다', 1.2895030515348342),
 ('원래', 1.285232866502736),
 ('붙다', 1.2785666918134482),
 ('놀라다', 1.2770738340532841),
 ('다음', 1.271585666979784),
 ('사진', 1.2701213502407973),
 ('생기다', 1.2666414178542353),
 ('내용물', 1.264680510515764),
 ('예전', 1.2509924226890723),
 ('이렇다', 1.2479397455077856),
 ('특별하다', 1.2405774817754203),
 ('골드', 1.2397137236709992),
 ('돼다', 1.2395581966752567),
 ('열다', 1.2392492035405136),
 ('쫀득', 1.2363302317725295),
 ('일반', 1.2343698673118524),
 ('저녁', 1.230880946311918),
 ('지도', 1.2198917539690166),
 ('기름', 1.2196013727480042),
 ('스킨', 1.2132589375438394),
 ('홈쇼핑', 1.2023012890243696),
 ('펴다', 1.1983013007133492),
 ('프리미엄', 1.197305455000642),
 ('그것', 1.1939703870891363),
 ('만족', 1.192319814416714),
 ('사보다', 1.1912554511609104),
 ('줄다', 1.1900091312685848),
 ('막다', 1.1828700956478693),
 ('묻히다', 1.1816298035897599),
 ('비교', 1.1778160336040742),
 ('다르다', 1.1758546527292235),
 ('꼼꼼하다', 1.1755338904937822),
 ('쿨링감', 1.1746675233445716),
 ('기도', 1.174646748848415),
 ('완전', 1.1729541007445121),
 ('만족스럽다', 1.1700725112531245),
 ('기미', 1.1653004786328278),
 ('안되다', 1.1612111764939848),
 ('자체', 1.1504958141455068),
 ('예요', 1.1504897285292937),
 ('이랑', 1.1503037795740656),
 ('으로는', 1.1413374792232975),
 ('뭔가', 1.1393786651274458),
 ('드네', 1.1375074623825445),
 ('이탈', 1.1338604621199726),
 ('우선', 1.1288532442274177),
 ('지성', 1.1282533529217493),
 ('여서', 1.1281384711637645),
 ('그대로', 1.1276097283959206),
 ('깨다', 1.1273215929852527),
 ('올라오다', 1.1258292532811476),
 ('기다', 1.124258709753131),
 ('다시다', 1.1192263005228305),
 ('차다', 1.1182300140516495),
 ('확실하다', 1.1181918208065817),
 ('마다', 1.114815923566831),
 ('없어지다', 1.11199709906117),
 ('사이즈', 1.0939268642119426),
 ('파우더', 1.0897735135723046),
 ('짜다', 1.0891964564897496),
 ('사실', 1.0826693357289248),
 ('가지', 1.082214328526657),
 ('겠다', 1.0801194320962186),
 ('느껴지다', 1.077468464121698),
 ('덥다', 1.0753568003182772),
 ('기르다', 1.073919634881938),
 ('그런', 1.0646948046800344),
 ('드리다', 1.064625933927462),
 ('약하다', 1.0637261613581388),
 ('골든', 1.0621729318842736),
 ('쿠폰', 1.0613918710197798),
 ('인가', 1.0605414774660207),
 ('에만', 1.057737933386762),
 ('은은하다', 1.0558216549722552),
 ('질감', 1.0470047911868512),
 ('누가', 1.0462642779718978),
 ('비닐', 1.0424611396771),
 ('이라서', 1.0401354352709964),
 ('참고', 1.035601363930196),
 ('전혀', 1.0309520644189332),
 ('싫어하다', 1.0205807872066561),
 ('치다', 1.0190338684388747),
 ('찌다', 1.0106713391039164),
 ('높다', 1.0094731781433273),
 ('주황색', 1.008415365831815),
 ('부탁드리다', 1.0080390289675414),
 ('보송하다', 1.0076961866231797),
 ('평타', 1.0)]

In [86]:
evi[0:20]

[('좋다', 227.22701361175888),
 ('피부', 163.9662699171092),
 ('사용', 140.39377985440854),
 ('바르다', 132.1719369246269),
 ('되다', 105.3657482606096),
 ('크림', 94.20150677556853),
 ('구매', 93.64971602243664),
 ('느낌', 81.61301076085618),
 ('촉촉하다', 79.73306517787624),
 ('받다', 78.76515950245205),
 ('선물', 72.5226332346559),
 ('가격', 65.82799053545432),
 ('들다', 63.99661638705449),
 ('배송', 62.60689054664835),
 ('후기', 62.14687572070677),
 ('체험', 60.94523849546171),
 ('보습', 60.09443488911877),
 ('화장품', 59.60066268001259),
 ('얼굴', 56.55716589302353),
 ('에센스', 53.483545942275335)]

In [87]:
no[0:20]

[('바르다', 17.557392019683295),
 ('좋다', 17.31293945475131),
 ('크림', 11.011995420569562),
 ('사용', 10.691748450610053),
 ('피부', 10.388048080632156),
 ('스틱', 9.2673474501341),
 ('되다', 8.490631538681068),
 ('가격', 7.6748752541066985),
 ('얼굴', 6.960933494174066),
 ('보습', 6.5816899457209725),
 ('촉촉하다', 5.549578823978562),
 ('괜찮다', 5.191804640856905),
 ('모르다', 5.177223154246008),
 ('쿠션', 5.163937531442285),
 ('편하다', 5.134836962627943),
 ('저렴하다', 4.6218439460229535),
 ('많이', 4.221461441605375),
 ('효과', 4.1088223189404856),
 ('구입', 4.024362544695146),
 ('크다', 3.9646751819985955)]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [16]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return title_to_index.keys()[movie_indices]

In [20]:
# title_to_index = dict(zip(['title'], data.index))
title_to_index = {}
for col in cols:
    title_to_index[col] = list(cols).index(col)

In [26]:
cosine_sim.shape

(17625, 17625)

In [21]:
# NoEvinue 피부 가 어떤 단어들과 같이 나오는지
get_recommendations("피부")

IndexError: index 19622 is out of bounds for axis 0 with size 17625

## NoEvinue 에서 피부 라는 단어와 자주 같이 나오는 단어들 찾아보기

In [14]:
tfidf_matrix.head(1)

,00,000,00000000001,00001,0005,000원,005,01,01024072031,010원,...,힘쓰다,힘안,힘없다,힘유,힘주다,힙듭니,ｊヽ,ｦｦｦ,ﾉﾞ,ﾟﾟ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
test1 = tfidf_matrix[tfidf_matrix['피부']>0.1]

In [16]:
test1.sort_values("피부", ascending=False)

,00,000,00000000001,00001,0005,000원,005,01,01024072031,010원,...,힘쓰다,힘안,힘없다,힘유,힘주다,힙듭니,ｊヽ,ｦｦｦ,ﾉﾞ,ﾟﾟ
6825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
ind = test1.index
ind

Int64Index([    9,    13,    14,    16,    17,    20,    21,    22,    23,
               29,
            ...
            17523, 17531, 17558, 17570, 17572, 17575, 17600, 17617, 17618,
            17620],
           dtype='int64', length=2947)

In [22]:
ind[9:12]

Int64Index([29, 30, 37], dtype='int64')

In [19]:
test1_dict = {}
for col in cols:
    test1_dict[col] = test1.loc[[9]][col].values
sorted(test1_dict.items(), key=lambda x: x[1], reverse=True)

[('구매', array([0.22294159])),
 ('후기', array([0.19143375])),
 ('크림', array([0.18749263])),
 ('어마', array([0.17921747])),
 ('서로', array([0.17571458])),
 ('참조', array([0.17000375])),
 ('탄력', array([0.16709523])),
 ('되다', array([0.16412113])),
 ('즐겁다', array([0.16410666])),
 ('쇼핑', array([0.14503905])),
 ('가성', array([0.14257111])),
 ('많다', array([0.13466762])),
 ('금액', array([0.13196457])),
 ('도움', array([0.12494541])),
 ('들다', array([0.1236641])),
 ('마지막', array([0.11661158])),
 ('윈윈된다', array([0.11457343])),
 ('바라다', array([0.10949952])),
 ('피부', array([0.10853753])),
 ('나이', array([0.10721126])),
 ('브랜드', array([0.1050358])),
 ('명가', array([0.10209108])),
 ('역시', array([0.10138862])),
 ('에스테', array([0.10033964])),
 ('줄기', array([0.10033964])),
 ('항상', array([0.09771612])),
 ('품류', array([0.09748422])),
 ('볼살', array([0.0942156])),
 ('희망', array([0.09025817])),
 ('입시', array([0.08960874])),
 ('컴플렉스', array([0.08960874])),
 ('잃다', array([0.08732873])),
 ('찾다', array([0.08716731])),
 ('리

In [53]:
test1_dict = {}
for col in cols:
    test1_dict[col] = test1.loc[[13]][col].values
sorted(test1_dict.items(), key=lambda x: x[1], reverse=True)

[('선사', array([0.27314655])),
 ('풍부하다', array([0.24602181])),
 ('크림', array([0.24076043])),
 ('콜라겐', array([0.1964336])),
 ('피부', array([0.19114108])),
 ('양감', array([0.19006264])),
 ('건조', array([0.16658656])),
 ('탄탄하다', array([0.15700383])),
 ('골드', array([0.15462])),
 ('들다', array([0.13611242])),
 ('치다', array([0.13419763])),
 ('있짢아', array([0.12610667])),
 ('보다', array([0.12264679])),
 ('탄력', array([0.12261029])),
 ('젊은이', array([0.12103607])),
 ('다재', array([0.11743842])),
 ('있다', array([0.11323244])),
 ('농밀', array([0.11236782])),
 ('뭐시기', array([0.11236782])),
 ('괜츈', array([0.11044006])),
 ('발해', array([0.1047877])),
 ('영양성분', array([0.1047877])),
 ('수분크림', array([0.10468363])),
 ('겉돈', array([0.10369957])),
 ('효율', array([0.10177181])),
 ('어쩌', array([0.09934377])),
 ('심하다', array([0.09919189])),
 ('소문나다', array([0.09731137])),
 ('시키다', array([0.09705467])),
 ('헤프', array([0.09556355])),
 ('인스타', array([0.09503132])),
 ('짙다', array([0.09503132])),
 ('부여', array([0.09183071])),

In [54]:
test1_dict = {}
for col in cols:
    test1_dict[col] = test1.loc[[14]][col].values
sorted(test1_dict.items(), key=lambda x: x[1], reverse=True)

[('콜라겐', array([0.26181784])),
 ('이든', array([0.2482075])),
 ('화장품', array([0.2059223])),
 ('피부', array([0.19107275])),
 ('제품', array([0.18617423])),
 ('제일', array([0.17335143])),
 ('관리', array([0.16024658])),
 ('이나마', array([0.15927518])),
 ('가꾸다', array([0.15897212])),
 ('꾸준하다', array([0.14583457])),
 ('크림', array([0.13752821])),
 ('도움', array([0.13747355])),
 ('되다', array([0.12898384])),
 ('까지미백', array([0.12606159])),
 ('여신', array([0.1209928])),
 ('탈골', array([0.11460688])),
 ('있다', array([0.11319196])),
 ('되어다', array([0.11281709])),
 ('유도체', array([0.10594173])),
 ('다니다', array([0.10591242])),
 ('후기', array([0.1053143])),
 ('영양성분', array([0.10475024])),
 ('가성', array([0.10457771])),
 ('사용', array([0.10318707])),
 ('프라', array([0.10173543])),
 ('효율', array([0.10173543])),
 ('바이', array([0.10006613])),
 ('좋다', array([0.0993368])),
 ('13일', array([0.09859371])),
 ('lg', array([0.09859371])),
 ('바램', array([0.09399672])),
 ('진주', array([0.09028073])),
 ('주다', array([0.08887243])),
 

In [55]:
test1_dict = {}
for col in cols:
    test1_dict[col] = test1.loc[[16]][col].values
sorted(test1_dict.items(), key=lambda x: x[1], reverse=True)

[('선사', array([0.30960285])),
 ('콜라겐', array([0.29686824])),
 ('풍부하다', array([0.27885783])),
 ('피부', array([0.21665228])),
 ('양감', array([0.2154299])),
 ('무난', array([0.19540075])),
 ('탄탄하다', array([0.1779588])),
 ('골드', array([0.17525681])),
 ('크림', array([0.15593956])),
 ('탄력', array([0.13897483])),
 ('일리', array([0.13538907])),
 ('농밀', array([0.12736531])),
 ('유통', array([0.12169002])),
 ('영양성분', array([0.11877349])),
 ('겉돈', array([0.11754013])),
 ('기한', array([0.11747493])),
 ('가격', array([0.11681602])),
 ('효율', array([0.11535508])),
 ('골든', array([0.11346231])),
 ('이탈', array([0.10960772])),
 ('짙다', array([0.10771495])),
 ('어쩌나', array([0.10455195])),
 ('부여', array([0.10408716])),
 ('기기', array([0.10277779])),
 ('전달', array([0.1023669])),
 ('진주', array([0.1023669])),
 ('이에요', array([0.10154624])),
 ('프리미엄', array([0.09978254])),
 ('저렴하다', array([0.09898573])),
 ('정말', array([0.09775006])),
 ('간혹', array([0.09703043])),
 ('농도', array([0.09119072])),
 ('짧다', array([0.09082725])),
 

In [56]:
test1_dict = {}
for col in cols:
    test1_dict[col] = test1.loc[[17]][col].values
sorted(test1_dict.items(), key=lambda x: x[1], reverse=True)

[('크림', array([0.28542686])),
 ('탄력', array([0.22611125])),
 ('콜라겐', array([0.1811261])),
 ('피부', array([0.17624601])),
 ('화장품', array([0.15195462])),
 ('까지', array([0.14915987])),
 ('03', array([0.14691585])),
 ('손바닥', array([0.14555082])),
 ('담날', array([0.14035168])),
 ('이다', array([0.12799232])),
 ('일자', array([0.12741549])),
 ('제품', array([0.12266257])),
 ('틀림없다', array([0.11160408])),
 ('좋다', array([0.10995423])),
 ('바르다', array([0.10889114])),
 ('듬뿜', array([0.10828678])),
 ('무배', array([0.10828678])),
 ('마무리', array([0.10689555])),
 ('이중기', array([0.10571369])),
 ('수용성', array([0.10361132])),
 ('듬뿍', array([0.10244415])),
 ('능성', array([0.10183379])),
 ('50', array([0.09694091])),
 ('가격', array([0.09502949])),
 ('손끝', array([0.09469558])),
 ('이러하다', array([0.09469558])),
 ('기초', array([0.09384126])),
 ('보름', array([0.09304547])),
 ('푸석푸석', array([0.09304547])),
 ('하고', array([0.09259479])),
 ('골든', array([0.09230127])),
 ('어김없이', array([0.0909431])),
 ('영양분', array([0.08916557]

In [81]:
test1_dict = {}
for col in cols:
    test1_dict[col] = test1.loc[[20]][col].values
sorted(test1_dict.items(), key=lambda x: x[1], reverse=True)

[('콜라겐', array([0.28278356])),
 ('골든', array([0.25939004])),
 ('골드', array([0.20033026])),
 ('있다', array([0.17604877])),
 ('피부', array([0.16509871])),
 ('도움', array([0.1583809])),
 ('08', array([0.15313154])),
 ('크림', array([0.14854113])),
 ('려고', array([0.14043325])),
 ('비싸다', array([0.13377404])),
 ('기초', array([0.1318589])),
 ('거치다', array([0.12501334])),
 ('공급', array([0.12443335])),
 ('바르다', array([0.12240473])),
 ('일자', array([0.11935664])),
 ('이름', array([0.11825418])),
 ('제품', array([0.11490434])),
 ('되다', array([0.11145004])),
 ('에탄', array([0.10892503])),
 ('함유', array([0.10841166])),
 ('진지하다', array([0.10454528])),
 ('꺾다', array([0.09902745])),
 ('쫀쫀한', array([0.09902745])),
 ('ｊヽ', array([0.09902745])),
 ('되어다', array([0.09748097])),
 ('수용성', array([0.09705805])),
 ('무더위', array([0.09539295])),
 ('기존', array([0.09524892])),
 ('구매', array([0.09420031])),
 ('변신', array([0.09267831])),
 ('하루', array([0.09167133])),
 ('기분', array([0.08963514])),
 ('아니다', array([0.08507121])),
 

In [82]:
test1_dict = {}
for col in cols:
    test1_dict[col] = test1.loc[[21]][col].values
sorted(test1_dict.items(), key=lambda x: x[1], reverse=True)

[('ahc', array([0.39578237])),
 ('아주', array([0.30233799])),
 ('크림', array([0.22964177])),
 ('가격', array([0.16383538])),
 ('구매', array([0.15603418])),
 ('이벤트', array([0.14129153])),
 ('좋다', array([0.13269644])),
 ('콜라겐', array([0.12490801])),
 ('정말', array([0.1233857])),
 ('생기', array([0.1231936])),
 ('쫀쫀', array([0.1211905])),
 ('112원', array([0.12028288])),
 ('탄력', array([0.11694797])),
 ('세이프', array([0.11544645])),
 ('중요하다', array([0.11483315])),
 ('피부', array([0.1139462])),
 ('사실', array([0.1101185])),
 ('보습', array([0.10920665])),
 ('착하다', array([0.10898812])),
 ('추가', array([0.10795482])),
 ('참여', array([0.10717851])),
 ('유통', array([0.10240273])),
 ('겸사겸사', array([0.10234208])),
 ('랍니다', array([0.10234208])),
 ('알맹이', array([0.10234208])),
 ('혜택', array([0.10108533])),
 ('5만원', array([0.09994846])),
 ('그야말로', array([0.09994846])),
 ('기한', array([0.09885571])),
 ('미니미', array([0.09795582])),
 ('들다', array([0.09736992])),
 ('쫀득해', array([0.09707185])),
 ('탱글탱글', array([0.09707185

In [83]:
test1_dict = {}
for col in cols:
    test1_dict[col] = test1.loc[[22]][col].values
sorted(test1_dict.items(), key=lambda x: x[1], reverse=True)

[('가격', array([0.27818171])),
 ('착하다', array([0.27758201])),
 ('콜마', array([0.24513645])),
 ('제품', array([0.23698789])),
 ('go', array([0.19019402])),
 ('건성', array([0.17692997])),
 ('피부', array([0.17412584])),
 ('크림', array([0.16710714])),
 ('콜라겐', array([0.1590642])),
 ('넘다', array([0.1584721])),
 ('구매', array([0.13246789])),
 ('좋다', array([0.12874848])),
 ('골드', array([0.12520519])),
 ('비타민', array([0.11378372])),
 ('쓰다', array([0.10870705])),
 ('하고', array([0.10842182])),
 ('촉촉', array([0.10580592])),
 ('이신', array([0.10496031])),
 ('아깝다', array([0.10302314])),
 ('370', array([0.10211622])),
 ('마제', array([0.10211622])),
 ('워낙', array([0.09584044])),
 ('제조업체', array([0.09283733])),
 ('탈골', array([0.09283733])),
 ('대비', array([0.09277333])),
 ('보습', array([0.09271286])),
 ('고막', array([0.09099104])),
 ('농밀', array([0.09099104])),
 ('수용성', array([0.09099104])),
 ('이라', array([0.08779503])),
 ('기능', array([0.08734257])),
 ('분들', array([0.08608876])),
 ('유도체', array([0.08581812])),
 ('

In [20]:
test1_dict = {}
for col in cols:
    test1_dict[col] = test1.loc[[29]][col].values
sorted(test1_dict.items(), key=lambda x: x[1], reverse=True)

[('ahc', array([0.44234246])),
 ('주름', array([0.24265895])),
 ('크림', array([0.2199917])),
 ('이용', array([0.21494603])),
 ('에도', array([0.20333491])),
 ('탄력', array([0.19605871])),
 ('도감', array([0.18778868])),
 ('있다', array([0.18106315])),
 ('골든', array([0.16006693])),
 ('꾸준하다', array([0.1555191])),
 ('이탈', array([0.15462907])),
 ('바닥', array([0.12917356])),
 ('비도', array([0.12813452])),
 ('형도', array([0.12763457])),
 ('제품', array([0.12763133])),
 ('한여름', array([0.12714681])),
 ('좋다', array([0.12712023])),
 ('에는', array([0.12530803])),
 ('날다', array([0.12011098])),
 ('도포', array([0.11817427])),
 ('편안하다', array([0.11802104])),
 ('같다', array([0.11533927])),
 ('피부', array([0.11461581])),
 ('이전', array([0.11352833])),
 ('이후', array([0.10738314])),
 ('히알루론산', array([0.10583087])),
 ('콜라겐', array([0.1047017])),
 ('급하다', array([0.10297481])),
 ('밖에', array([0.10254781])),
 ('쓰이다', array([0.10219882])),
 ('쫀쫀', array([0.10158557])),
 ('으로도', array([0.09709654])),
 ('물론', array([0.09641147])),


In [23]:
test1_dict = {}
for col in cols:
    test1_dict[col] = test1.loc[[30]][col].values
sorted(test1_dict.items(), key=lambda x: x[1], reverse=True)

[('ahc', array([0.25758023])),
 ('크림', array([0.25620653])),
 ('수두', array([0.20566944])),
 ('비싸다', array([0.19227999])),
 ('선전', array([0.18792004])),
 ('겨울', array([0.18703587])),
 ('봄철', array([0.18500512])),
 ('든지', array([0.17697417])),
 ('부터', array([0.17424674])),
 ('때문', array([0.16455487])),
 ('이야기', array([0.14922791])),
 ('좋다', array([0.14804664])),
 ('강화', array([0.14725069])),
 ('에는', array([0.14593612])),
 ('인터넷', array([0.13949212])),
 ('대박', array([0.13835585])),
 ('피부', array([0.13348376])),
 ('편리하다', array([0.12736382])),
 ('들다', array([0.12673918])),
 ('가족', array([0.12444213])),
 ('바뀌다', array([0.12354437])),
 ('순간', array([0.12315621])),
 ('콜라겐', array([0.1219376])),
 ('필수', array([0.1207796])),
 ('종류', array([0.12060644])),
 ('쓰이다', array([0.11902269])),
 ('판매', array([0.11326855])),
 ('안좋다', array([0.11295595])),
 ('미백', array([0.11252523])),
 ('중요하다', array([0.11210233])),
 ('무겁다', array([0.11087785])),
 ('어느', array([0.11031664])),
 ('나이', array([0.10987722])),

## Evinue

In [57]:
prod_list = os.listdir("./Evinue/")
review_content1 = []
for name in prod_list[0:300]:
#     name = prod_list[0]
    prod_name = name.split(",")[0].split(" ")
    set(prod_name)
    
    df = pd.read_excel(f"./Evinue/{name}")
    test = df["리뷰 내용"].values.tolist() # name 이라는 상품의 모든 리뷰가 담겨있음
    new_review = make_review(test, prod_name)
    review_content1 += new_review

In [58]:
# Evinue
text1 = []
for i in range(len(review_content1)):
    test = review_content1[i]
#     test.to_list()
#     test = test.split(",")
    test = " ".join(test)
    text1.append(test)

In [59]:
# Evinue
tfidfv = TfidfVectorizer().fit(text1)
print(tfidfv.transform(text1).toarray())
print()
print(tfidfv.vocabulary_)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

{'강하다': 883, '파우더': 9446, '이구': 7199, '딱하다': 2875, '호불호': 10173, '수가': 5312, '없다': 6321, '그냥': 1473, '이라는': 7248, '브렌': 4558, '믿다': 3873, '쓰다': 5804, '은은하다': 7136, '기분': 1631, '좋다': 8118, '아침': 5934, '에까지': 6346, '수분': 5339, '탱탱하다': 9240, '있다': 7566, '개인': 922, '피부': 9757, '타입': 9196, '건성': 1018, '흡수': 10373, '묽다': 3787, '워터': 6964, '리해': 3203, '언니': 6270, '화장품': 10237, '냄새': 2022, '느낌': 2196, '알쥬': 6035, '인공': 7404, '평전': 9606, '성분': 5128, '보다': 4267, '여러': 6452, '정제': 7977, '되다': 2689, '식물성': 5620, '들이다': 2811, '자다': 7590, '배합': 4108, '되어다': 2695, '같다': 889, '호호': 10185, '아르': 5865, '타마': 9184, '섞이다': 5066, '비정': 4629, '들다': 2798, '단독': 2314, '사용': 4794, '하고': 9812, '때문': 2895, '확실하다': 10249, '차이': 8649, '느껴지다': 2192, '여러가지': 6453, '오히려': 6706, '겉돈': 1050, '달다': 2343, '방법': 4072, '발라': 4024, '우와': 6927, '없어지다': 6324, '않다': 6006, 

In [64]:
reverse_ = {v:k for k,v in tfidfv.vocabulary_.items()}

col = [0]*len(reverse_)
for i in range(len(reverse_)):
    col[i] = reverse_[i]

In [65]:
# 데이터 프레임으로 바꿔주기
data = tfidfv.transform(text1).toarray()
tfidf_matrix = pd.DataFrame(data, columns = col)

In [67]:
evinue_col = tfidf_matrix.columns

In [68]:
test2 = tfidf_matrix[tfidf_matrix['피부']>0.1]
test2.sort_values("피부", ascending=False)

,00,000,000원,00시,01,01월,01일,02,020원,024원,...,힘드다,힘들다,힘들엇어,힘쓰다,힘없다,힘주다,힙니,ﻬﻬﻬﻬﻬ,ｊヽ,ﾟωﾟ
248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.063693,0.0,0.0,0.0,0.0,0.0
827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [70]:
ind2 = test2.index
ind2

Int64Index([   5,   11,   12,   14,   17,   19,   20,   28,   38,   44,
            ...
            3340, 3343, 3347, 3348, 3351, 3357, 3358, 3359, 3361, 3364],
           dtype='int64', length=719)

In [71]:
test2_dict = {}
for col in evinue_col:
    test2_dict[col] = test2.loc[[5]][col].values
sorted(test2_dict.items(), key=lambda x: x[1], reverse=True)

[('동생', array([0.47658351])),
 ('금고', array([0.28726824])),
 ('두드러기', array([0.25379471])),
 ('말씀드리다', array([0.24526282])),
 ('오르다', array([0.24047721])),
 ('잇다', array([0.21074168])),
 ('쓰시', array([0.20078976])),
 ('신분', array([0.19628633])),
 ('이신', array([0.19421907])),
 ('느낌', array([0.17443202])),
 ('용감', array([0.17272646])),
 ('민감하다', array([0.16358765])),
 ('금방', array([0.1604116])),
 ('분들', array([0.15773674])),
 ('보다는', array([0.15404561])),
 ('늘다', array([0.1524379])),
 ('선택', array([0.14858979])),
 ('피부', array([0.13877734])),
 ('스며들다', array([0.13830881])),
 ('건성', array([0.12754314])),
 ('성분', array([0.12493649])),
 ('맞다', array([0.11581753])),
 ('많이', array([0.10754544])),
 ('하고', array([0.09794417])),
 ('자다', array([0.08507143])),
 ('사용', array([0.0728743])),
 ('받다', array([0.07284623])),
 ('보다', array([0.07155001])),
 ('같다', array([0.0711454])),
 ('있다', array([0.06643088])),
 ('좋다', array([0.05481959])),
 ('00', array([0.])),
 ('000', array([0.])),
 ('000원', array([0

In [72]:
test2_dict = {}
for col in evinue_col:
    test2_dict[col] = test2.loc[[11]][col].values
sorted(test2_dict.items(), key=lambda x: x[1], reverse=True)

[('그대로', array([0.31801975])),
 ('얹히다', array([0.26478714])),
 ('손실', array([0.25529387])),
 ('200', array([0.20754675])),
 ('없다', array([0.20415181])),
 ('옅다', array([0.20218376])),
 ('힘드다', array([0.18992248])),
 ('얼른', array([0.18880375])),
 ('바르다', array([0.18853986])),
 ('피부', array([0.17966018])),
 ('이다', array([0.17652544])),
 ('환하다', array([0.17398877])),
 ('담다', array([0.16380076])),
 ('돌다', array([0.15366919])),
 ('앰플', array([0.15218583])),
 ('느낌', array([0.15054566])),
 ('민감하다', array([0.1411863])),
 ('이지만', array([0.13681237])),
 ('예전', array([0.13569365])),
 ('남다', array([0.13136982])),
 ('고급', array([0.13098611])),
 ('작다', array([0.12931255])),
 ('전혀', array([0.12789301])),
 ('재다', array([0.12459034])),
 ('지다', array([0.12087866])),
 ('스며들다', array([0.11936908])),
 ('주름', array([0.11629718])),
 ('꾸준하다', array([0.11545255])),
 ('자극', array([0.11371229])),
 ('비싸다', array([0.11271055])),
 ('건성', array([0.11007765])),
 ('용량', array([0.10917897])),
 ('이나', array([0.10820834])

In [73]:
test2_dict = {}
for col in evinue_col:
    test2_dict[col] = test2.loc[[12]][col].values
sorted(test2_dict.items(), key=lambda x: x[1], reverse=True)

[('겨를', array([0.24249393])),
 ('부대끼다', array([0.24249393])),
 ('불앞', array([0.24249393])),
 ('요리사', array([0.24249393])),
 ('극도', array([0.22255401])),
 ('보다', array([0.21533109])),
 ('피하', array([0.19619487])),
 ('보드랍다', array([0.19345306])),
 ('일해', array([0.19345306])),
 ('뜨겁다', array([0.19094998])),
 ('요것', array([0.19094998])),
 ('아들', array([0.18267417])),
 ('허다', array([0.18267417])),
 ('몇개', array([0.16983572])),
 ('스킨', array([0.16918484])),
 ('키우다', array([0.16015635])),
 ('에서', array([0.15090602])),
 ('대신', array([0.15047698])),
 ('친구', array([0.14876753])),
 ('화장품', array([0.14705781])),
 ('매끄럽다', array([0.13673696])),
 ('부족하다', array([0.13143145])),
 ('물론', array([0.12695373])),
 ('찾다', array([0.11607782])),
 ('끄다', array([0.11572484])),
 ('진정', array([0.11119029])),
 ('바르다', array([0.10957386])),
 ('겨울', array([0.10688452])),
 ('산뜻하다', array([0.10537672])),
 ('피부', array([0.10441325])),
 ('적임', array([0.10280233])),
 ('엄마', array([0.09903104])),
 ('저렴하다', array([0.099031

In [74]:
test2_dict = {}
for col in evinue_col:
    test2_dict[col] = test2.loc[[14]][col].values
sorted(test2_dict.items(), key=lambda x: x[1], reverse=True)

[('좋다', array([0.40489811])),
 ('민감', array([0.33124943])),
 ('순하다', array([0.28273404])),
 ('원래', array([0.27823042])),
 ('피부', array([0.25625275])),
 ('스며들다', array([0.2553876])),
 ('이고', array([0.25202502])),
 ('저렴하다', array([0.24304365])),
 ('가볍다', array([0.23550877])),
 ('아주', array([0.22676136])),
 ('인데', array([0.18755322])),
 ('에서', array([0.18517804])),
 ('촉촉하다', array([0.16428588])),
 ('자다', array([0.15708464])),
 ('쿠팡', array([0.13534445])),
 ('사용', array([0.13456259])),
 ('받다', array([0.13451078])),
 ('제품', array([0.13322755])),
 ('구매', array([0.12336907])),
 ('00', array([0.])),
 ('000', array([0.])),
 ('000원', array([0.])),
 ('00시', array([0.])),
 ('01', array([0.])),
 ('01월', array([0.])),
 ('01일', array([0.])),
 ('02', array([0.])),
 ('020원', array([0.])),
 ('024원', array([0.])),
 ('02월', array([0.])),
 ('02일', array([0.])),
 ('03', array([0.])),
 ('035', array([0.])),
 ('03월', array([0.])),
 ('03일', array([0.])),
 ('04', array([0.])),
 ('04월', array([0.])),
 ('05', arr

In [75]:
test2_dict = {}
for col in evinue_col:
    test2_dict[col] = test2.loc[[17]][col].values
sorted(test2_dict.items(), key=lambda x: x[1], reverse=True)

[('한데', array([0.41759364])),
 ('스럽지', array([0.38835252])),
 ('부담', array([0.29372094])),
 ('않다', array([0.28774505])),
 ('완전', array([0.28502156])),
 ('예민하다', array([0.28142493])),
 ('워터', array([0.27950168])),
 ('이지', array([0.27580254])),
 ('자극', array([0.24851065])),
 ('정도', array([0.21162182])),
 ('보습', array([0.20258666])),
 ('피부', array([0.13087846])),
 ('있다', array([0.12529959])),
 ('좋다', array([0.10339878])),
 ('00', array([0.])),
 ('000', array([0.])),
 ('000원', array([0.])),
 ('00시', array([0.])),
 ('01', array([0.])),
 ('01월', array([0.])),
 ('01일', array([0.])),
 ('02', array([0.])),
 ('020원', array([0.])),
 ('024원', array([0.])),
 ('02월', array([0.])),
 ('02일', array([0.])),
 ('03', array([0.])),
 ('035', array([0.])),
 ('03월', array([0.])),
 ('03일', array([0.])),
 ('04', array([0.])),
 ('04월', array([0.])),
 ('05', array([0.])),
 ('050원', array([0.])),
 ('05월', array([0.])),
 ('05일', array([0.])),
 ('06', array([0.])),
 ('060원', array([0.])),
 ('06월', array([0.])),
 ('0

In [76]:
test2_dict = {}
for col in evinue_col:
    test2_dict[col] = test2.loc[[19]][col].values
sorted(test2_dict.items(), key=lambda x: x[1], reverse=True)

[('딱좋다', array([0.52107954])),
 ('루지', array([0.514796])),
 ('가라앉다', array([0.44914861])),
 ('예민하다', array([0.30634049])),
 ('보이다', array([0.28590766])),
 ('인데', array([0.20854319])),
 ('바르다', array([0.14950694])),
 ('피부', array([0.1424656])),
 ('00', array([0.])),
 ('000', array([0.])),
 ('000원', array([0.])),
 ('00시', array([0.])),
 ('01', array([0.])),
 ('01월', array([0.])),
 ('01일', array([0.])),
 ('02', array([0.])),
 ('020원', array([0.])),
 ('024원', array([0.])),
 ('02월', array([0.])),
 ('02일', array([0.])),
 ('03', array([0.])),
 ('035', array([0.])),
 ('03월', array([0.])),
 ('03일', array([0.])),
 ('04', array([0.])),
 ('04월', array([0.])),
 ('05', array([0.])),
 ('050원', array([0.])),
 ('05월', array([0.])),
 ('05일', array([0.])),
 ('06', array([0.])),
 ('060원', array([0.])),
 ('06월', array([0.])),
 ('06일', array([0.])),
 ('07', array([0.])),
 ('075', array([0.])),
 ('07분', array([0.])),
 ('08', array([0.])),
 ('080', array([0.])),
 ('08월', array([0.])),
 ('08일', array([0.])),
 

In [77]:
test2_dict = {}
for col in evinue_col:
    test2_dict[col] = test2.loc[[20]][col].values
sorted(test2_dict.items(), key=lambda x: x[1], reverse=True)

[('잘맞다', array([0.64061675])),
 ('완전', array([0.45935234])),
 ('구입', array([0.41444855])),
 ('다음', array([0.402923])),
 ('피부', array([0.21092905])),
 ('00', array([0.])),
 ('000', array([0.])),
 ('000원', array([0.])),
 ('00시', array([0.])),
 ('01', array([0.])),
 ('01월', array([0.])),
 ('01일', array([0.])),
 ('02', array([0.])),
 ('020원', array([0.])),
 ('024원', array([0.])),
 ('02월', array([0.])),
 ('02일', array([0.])),
 ('03', array([0.])),
 ('035', array([0.])),
 ('03월', array([0.])),
 ('03일', array([0.])),
 ('04', array([0.])),
 ('04월', array([0.])),
 ('05', array([0.])),
 ('050원', array([0.])),
 ('05월', array([0.])),
 ('05일', array([0.])),
 ('06', array([0.])),
 ('060원', array([0.])),
 ('06월', array([0.])),
 ('06일', array([0.])),
 ('07', array([0.])),
 ('075', array([0.])),
 ('07분', array([0.])),
 ('08', array([0.])),
 ('080', array([0.])),
 ('08월', array([0.])),
 ('08일', array([0.])),
 ('09', array([0.])),
 ('090원', array([0.])),
 ('09월', array([0.])),
 ('09일', array([0.])),
 ('0

In [78]:
test2_dict = {}
for col in evinue_col:
    test2_dict[col] = test2.loc[[28]][col].values
sorted(test2_dict.items(), key=lambda x: x[1], reverse=True)

[('민감', array([0.89694442])),
 ('피부', array([0.34693565])),
 ('좋다', array([0.27409187])),
 ('00', array([0.])),
 ('000', array([0.])),
 ('000원', array([0.])),
 ('00시', array([0.])),
 ('01', array([0.])),
 ('01월', array([0.])),
 ('01일', array([0.])),
 ('02', array([0.])),
 ('020원', array([0.])),
 ('024원', array([0.])),
 ('02월', array([0.])),
 ('02일', array([0.])),
 ('03', array([0.])),
 ('035', array([0.])),
 ('03월', array([0.])),
 ('03일', array([0.])),
 ('04', array([0.])),
 ('04월', array([0.])),
 ('05', array([0.])),
 ('050원', array([0.])),
 ('05월', array([0.])),
 ('05일', array([0.])),
 ('06', array([0.])),
 ('060원', array([0.])),
 ('06월', array([0.])),
 ('06일', array([0.])),
 ('07', array([0.])),
 ('075', array([0.])),
 ('07분', array([0.])),
 ('08', array([0.])),
 ('080', array([0.])),
 ('08월', array([0.])),
 ('08일', array([0.])),
 ('09', array([0.])),
 ('090원', array([0.])),
 ('09월', array([0.])),
 ('09일', array([0.])),
 ('0원', array([0.])),
 ('10', array([0.])),
 ('100', array([0.]

In [79]:
test2_dict = {}
for col in evinue_col:
    test2_dict[col] = test2.loc[[38]][col].values
sorted(test2_dict.items(), key=lambda x: x[1], reverse=True)

[('역할', array([0.37990965])),
 ('바닐라', array([0.18657536])),
 ('소화기', array([0.18657536])),
 ('코렉터', array([0.18657536])),
 ('나용', array([0.17760097])),
 ('초초', array([0.17123354])),
 ('잠재우다', array([0.16629458])),
 ('화끈하다', array([0.16225915])),
 ('민트', array([0.15884725])),
 ('싼값', array([0.15328476])),
 ('신기', array([0.15095276])),
 ('균일하다', array([0.14691733])),
 ('성능', array([0.14350543])),
 ('비누', array([0.14197837])),
 ('안정', array([0.14197837])),
 ('해주다', array([0.14111361])),
 ('놀래다', array([0.13920806])),
 ('색감', array([0.13674624])),
 ('빨갛다', array([0.13561094])),
 ('아토피', array([0.13561094])),
 ('유명', array([0.13453104])),
 ('인생', array([0.12738691])),
 ('보다', array([0.12425716])),
 ('생각', array([0.12403207])),
 ('같다', array([0.1235545])),
 ('피부', array([0.12050367])),
 ('이어서', array([0.11866841])),
 ('아무리', array([0.11533016])),
 ('붉다', array([0.11166673])),
 ('섞다', array([0.11129473])),
 ('크기', array([0.10788283])),
 ('야하다', array([0.10520571])),
 ('민감', array([0.1038474]

In [80]:
test2_dict = {}
for col in evinue_col:
    test2_dict[col] = test2.loc[[44]][col].values
sorted(test2_dict.items(), key=lambda x: x[1], reverse=True)

[('여보', array([0.50554289])),
 ('준템', array([0.50554289])),
 ('반짝', array([0.42240278])),
 ('매끈하다', array([0.28660229])),
 ('여서', array([0.26857531])),
 ('꾸다', array([0.26466871])),
 ('화장', array([0.19718237])),
 ('발라', array([0.16728774])),
 ('피부', array([0.10883855])),
 ('좋다', array([0.08598644])),
 ('00', array([0.])),
 ('000', array([0.])),
 ('000원', array([0.])),
 ('00시', array([0.])),
 ('01', array([0.])),
 ('01월', array([0.])),
 ('01일', array([0.])),
 ('02', array([0.])),
 ('020원', array([0.])),
 ('024원', array([0.])),
 ('02월', array([0.])),
 ('02일', array([0.])),
 ('03', array([0.])),
 ('035', array([0.])),
 ('03월', array([0.])),
 ('03일', array([0.])),
 ('04', array([0.])),
 ('04월', array([0.])),
 ('05', array([0.])),
 ('050원', array([0.])),
 ('05월', array([0.])),
 ('05일', array([0.])),
 ('06', array([0.])),
 ('060원', array([0.])),
 ('06월', array([0.])),
 ('06일', array([0.])),
 ('07', array([0.])),
 ('075', array([0.])),
 ('07분', array([0.])),
 ('08', array([0.])),
 ('080', arr